<a href="https://colab.research.google.com/github/Epalaciol/VisionArtificial_2021_Retinas/blob/master/TrabajoRetinas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Vision Artificial Retinas

##Integrantes: 
- Alejandra Betancur Palacio
- Juan Camilo  Molina Roncancio
- Esteban Palacio Londoño
- Valentina Tamayo Guarin 
- Mateo Valencia Diaz
- Angie Tatiana Solano Rodriguez


## Introducción

[Inserte aqui la intro]


## Preprocesamiento de imagenes

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
from skimage import io, color

ModuleNotFoundError: No module named 'cv2'

In [ ]:
imagenUno = cv2.imread("images/training/images/21_training.tif", cv2.IMREAD_GRAYSCALE)
plt.imshow(imagenUno, cmap= "gray")

#Segmentación 